In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 6.4 MB/s 
     |████████████████████████████████| 81 kB 8.2 MB/s 
     |████████████████████████████████| 209 kB 66.1 MB/s 
     |████████████████████████████████| 78 kB 6.5 MB/s 
     |████████████████████████████████| 147 kB 48.1 MB/s 
     |████████████████████████████████| 112 kB 42.3 MB/s 
     |████████████████████████████████| 50 kB 4.3 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=f85a1d321b7d59cf1da7e2266e2a51c7600eff8e0132db0140b04bd058e00e32
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [47]:
import optuna

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.ensemble import VotingClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import joblib

In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
base_path = '/content/drive/MyDrive/Colab Notebooks/my_project/'
df_train = pd.read_csv(base_path  +  'train.txt', encoding='utf-8')

data = pd.read_csv("/content/total.csv")

In [ ]:
# gender 3개 범주로 만들어주기
data['gender'] = data['gender'].apply(lambda x: 3 if x >= 3 else x)

# product_name 지우기
data = data.iloc[:,1:]

# rating 0인거 지우기 (이진분류) #해주면 3진 분류로 가능
data = data.loc[data.loc[:,'rating'] != 0,:]

In [ ]:
# 2) null 데이터 체크 함수
# Age, Cabin, Embarked, Fare
# 입력 : df --> 누락처리 --> 출력:df

def check_fillna(df):
    df["brand_name"].fillna("RISINGWAVE", inplace=True)
    return df

In [ ]:
# 불필요한 컬럼 제거 함수
# df --> 컬럼 제거 --> df

def drop_features(df):
    df.drop(["goodsnum"], axis=1, inplace=True)
    return df

In [ ]:
def encode_feature(df):
    # 인코딩 컬럼을 리스트업
    features = ["brand_name", "season", "exit"]
    # 위의 컬럼들을 인코딩
    for f in features:
        le = LabelEncoder()
        le.fit(df.loc[:, f])
        df.loc[:, f] = le.transform(df.loc[:,f])
    return df

In [ ]:
### 전처리를 함수화 프로세스로

def musinsa_preprocess(df):
    df = check_fillna(df)
    df = drop_features(df)
    df = encode_feature(df)
    return df
#### ######

In [ ]:
y_data = data.loc[:, "rating"]  # like, rating범주화 별로 따로 진행
X_data = data.drop(columns = ["rating"])
X_data.info()
y_data = y_data - 1

<class 'pandas.core.frame.DataFrame'>
Int64Index: 703 entries, 0 to 1463
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   brand_name  701 non-null    object
 1   season      703 non-null    object
 2   gender      703 non-null    int64 
 3   review_cnt  703 non-null    int64 
 4   exit        703 non-null    object
 5   goodsnum    703 non-null    int64 
 6   price       703 non-null    int64 
dtypes: int64(4), object(3)
memory usage: 43.9+ KB


In [ ]:
X_data = musinsa_preprocess(X_data)
X_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 703 entries, 0 to 1463
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   brand_name  703 non-null    int64
 1   season      703 non-null    int64
 2   gender      703 non-null    int64
 3   review_cnt  703 non-null    int64
 4   exit        703 non-null    int64
 5   price       703 non-null    int64
dtypes: int64(6)
memory usage: 38.4 KB


In [ ]:
#### train_test split & kfold 선언
X_train, X_test, y_train, y_test = train_test_split(
      X_data, y_data, test_size = 0.2, random_state=1111 # 만약 범주화의 경우 stratify
)
kfold = KFold(n_splits = 5, random_state=1111, shuffle=True)

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
lgbm = LGBMClassifier(
    random_state = 1111,
    n_jobs = -1
)

In [ ]:
scores= cross_val_score(lgbm, X_train, y_train,
                        cv = kfold, scoring = "accuracy")

for iter_count, acc in enumerate(scores):
  print("LGBM {0}번째 검증 ACC : {1:.4f}".format(iter_count+1, acc))
print("LGBM 평균 ACC : ", scores.mean())

LGBM 1번째 검증 ACC : 0.6549
LGBM 2번째 검증 ACC : 0.6726
LGBM 3번째 검증 ACC : 0.6339
LGBM 4번째 검증 ACC : 0.6875
LGBM 5번째 검증 ACC : 0.7321
LGBM 평균 ACC :  0.6762010113780026


In [ ]:
def lgbm_objective(trial):
  # 1 ) Test Parameter : 가변적으로 test HPT의 값들을 셋팅
  params = {
      "boosting_type " : trial.suggest_categorical("boosting_type", ["gbdt"]),
      "num_leaves " : trial.suggest_int("num_leaves", 10, 150),
      "max_depth " : trial.suggest_int("max_depth", 1, 100),
      "learning_rate " : trial.suggest_float("learning_rate", 0.00001, 0.3),
      "n_estimators " : trial.suggest_int("n_estimators", 10, 1000),
      "subsample " : trial.suggest_float("subsample", 0.1, 15),
      "reg_alpha " : trial.suggest_float("reg_alpha", 0.1, 10) ,
      "reg_lambda  " : trial.suggest_float("reg_lambda ", 0.1, 10)
  }
  # 2 ) 1번에서 파라미터를 받아줄 모델 작성
  lgbm = LGBMClassifier(n_jobs = -1,
                      random_state = 1111, **params)
  # 3 ) 모델에 대해서 학습
  lgbm.fit(X_train, y_train)
  # 4 ) CV
  scores = cross_val_score(lgbm, X_train, y_train, cv = kfold, scoring = "accuracy")
  acc_mean = scores.mean()
  # 5 ) optimizer에 대한 target --> acc_mean의 스코어
  return acc_mean

In [ ]:
lgbm_study = optuna.create_study(direction = "maximize")
lgbm_study.optimize(lgbm_objective, n_trials = 50)

[I 2022-11-17 11:24:00,891] A new study created in memory with name: no-name-15ee4282-528d-4a1b-881d-9e03b11ee124
[I 2022-11-17 11:24:01,201] Trial 0 finished with value: 0.6762010113780026 and parameters: {'boosting_type': 'gbdt', 'num_leaves': 61, 'max_depth': 25, 'learning_rate': 0.15653784284285843, 'n_estimators': 194, 'subsample': 2.340819510067158, 'reg_alpha': 4.309486047989226, 'reg_lambda ': 0.34309418083044957}. Best is trial 0 with value: 0.6762010113780026.
[I 2022-11-17 11:24:01,481] Trial 1 finished with value: 0.6762010113780026 and parameters: {'boosting_type': 'gbdt', 'num_leaves': 82, 'max_depth': 96, 'learning_rate': 0.056835537392952766, 'n_estimators': 84, 'subsample': 11.849222842765904, 'reg_alpha': 9.970380712000726, 'reg_lambda ': 0.799344284644745}. Best is trial 0 with value: 0.6762010113780026.
[I 2022-11-17 11:24:01,768] Trial 2 finished with value: 0.6762010113780026 and parameters: {'boosting_type': 'gbdt', 'num_leaves': 43, 'max_depth': 45, 'learning_ra

KeyboardInterrupt: ignored